In [1]:
import geopandas as gp
import pandas as pd
import folium
import numpy as np

In [2]:
wac = gp.read_file('C:/Workspace/TNC-Demand-Model/Outputs/WAC_2019.shp')

In [3]:
rac = gp.read_file('C:/Workspace/TNC-Demand-Model/Outputs/RAC_2019.shp')

In [4]:
chi_tracts = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Chicago Tracts/geo_export_558aad9f-98d8-4dd5-a6b1-c1730155d596.shp')

In [5]:
chi_tracts.head()

,commarea,commarea_n,countyfp10,geoid10,name10,namelsad10,notes,statefp10,tractce10,geometry
0,44,44.0,031,17031842400,8424,Census Tract 8424,None,17,842400,"POLYGON ((-87.62405 41.73022, -87.62405 41.730..."
1,59,59.0,031,17031840300,8403,Census Tract 8403,None,17,840300,"POLYGON ((-87.68608 41.82296, -87.68607 41.823..."
2,34,34.0,031,17031841100,8411,Census Tract 8411,None,17,841100,"POLYGON ((-87.62935 41.85280, -87.62934 41.852..."
3,31,31.0,031,17031841200,8412,Census Tract 8412,None,17,841200,"POLYGON ((-87.68813 41.85569, -87.68816 41.856..."
4,32,32.0,031,17031839000,8390,Census Tract 8390,None,17,839000,"POLYGON ((-87.63312 41.87449, -87.63306 41.874..."


In [6]:
chi_tracts['TRACT'] = chi_tracts.geoid10.apply(lambda x: x[5:])

In [7]:
chi_wac = chi_tracts.merge(wac, how = 'left', left_on = 'TRACT', right_on = 'CENSUS_TRA')
chi_rac = chi_tracts.merge(rac, how = 'left', left_on = 'TRACT', right_on = 'CENSUS_TRA')

In [8]:
chi_wac = gp.GeoDataFrame(chi_wac, geometry = 'geometry_x')
chi_rac = gp.GeoDataFrame(chi_rac, geometry = 'geometry_x')

In [9]:
chi_tracts.head()

,commarea,commarea_n,countyfp10,geoid10,name10,namelsad10,notes,statefp10,tractce10,geometry,TRACT
0,44,44.0,031,17031842400,8424,Census Tract 8424,None,17,842400,"POLYGON ((-87.62405 41.73022, -87.62405 41.730...",842400
1,59,59.0,031,17031840300,8403,Census Tract 8403,None,17,840300,"POLYGON ((-87.68608 41.82296, -87.68607 41.823...",840300
2,34,34.0,031,17031841100,8411,Census Tract 8411,None,17,841100,"POLYGON ((-87.62935 41.85280, -87.62934 41.852...",841100
3,31,31.0,031,17031841200,8412,Census Tract 8412,None,17,841200,"POLYGON ((-87.68813 41.85569, -87.68816 41.856...",841200
4,32,32.0,031,17031839000,8390,Census Tract 8390,None,17,839000,"POLYGON ((-87.63312 41.87449, -87.63306 41.874...",839000


In [10]:
geo_area = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Tracts/tl_2019_17_tract/tl_2019_17_tract.shp')

geo_area['SQUARE_MILES_LAND'] = geo_area['ALAND']*3.86102e-7

geo_area['GEOID_STR'] = geo_area['GEOID'].astype(str)
chi_geo_area = chi_tracts.merge(geo_area, how = 'left', left_on = 'geoid10', right_on = 'GEOID_STR')
#chi_geo_area['GEOID_TRACT'] = chi_geo_area.apply(lambda row: row['GEOID_STR'][0][5:], axis = 1)

chi_wac = chi_wac.merge(chi_geo_area[['TRACT','SQUARE_MILES_LAND']], how = 'left', on = 'TRACT')
chi_rac = chi_rac.merge(chi_geo_area[['TRACT','SQUARE_MILES_LAND']], how = 'left', on = 'TRACT')


In [11]:
chi_wac.head()

,commarea,commarea_n,countyfp10,geoid10,name10,namelsad10,notes,statefp10,tractce10,geometry_x,...,TOTAL_EMP,29U_EMP,30_54_EMP,55P_EMP,1250U_EMP,1251_3333_,333P_EMP,FOOD_SERVI,geometry_y,SQUARE_MILES_LAND
0,44,44.0,031,17031842400,8424,Census Tract 8424,None,17,842400,"POLYGON ((-87.62405 41.73022, -87.62405 41.730...",...,4280.815406,1317.328678,2024.261548,939.225179,1413.865741,1756.773435,1110.176229,272.516085,"POLYGON ((-87.63933 41.75053, -87.63932 41.750...",0.760073
1,59,59.0,031,17031840300,8403,Census Tract 8403,None,17,840300,"POLYGON ((-87.68608 41.82296, -87.68607 41.823...",...,553.076926,127.710490,284.583218,140.783217,147.822378,216.202798,189.051749,162.906295,"POLYGON ((-87.68745 41.83463, -87.68696 41.834...",0.320870
2,34,34.0,031,17031841100,8411,Census Tract 8411,None,17,841100,"POLYGON ((-87.62935 41.85280, -87.62934 41.852...",...,3114.325889,589.278325,1605.934274,919.113291,1374.647559,1309.283923,430.394408,1287.160845,"POLYGON ((-87.64396 41.84947, -87.64325 41.849...",0.431409
3,31,31.0,031,17031841200,8412,Census Tract 8412,None,17,841200,"POLYGON ((-87.68813 41.85569, -87.68816 41.856...",...,668.720283,172.962239,362.013988,133.744057,142.794406,247.376225,278.549652,163.911889,"POLYGON ((-87.68822 41.85934, -87.68807 41.859...",0.241471
4,32,32.0,031,17031839000,8390,Census Tract 8390,None,17,839000,"POLYGON ((-87.63312 41.87449, -87.63306 41.874...",...,2239.458752,689.837766,1162.467139,387.153848,575.200003,827.604200,836.654550,201.118882,"POLYGON ((-87.63603 41.87445, -87.63592 41.874...",0.187528


In [12]:
chi_wac['TOTAL_EMP_PER_SQ_MILE'] = chi_wac['TOTAL_EMP'] / chi_wac['SQUARE_MILES_LAND']
chi_rac['TOTAL_WORKERS_PER_SQ_MILE'] = chi_rac['TOTAL_EMP'] / chi_rac['SQUARE_MILES_LAND']

In [13]:
centroids = pd.read_csv('C:/Workspace/TNC-Demand-Model/otp/points.csv')

centroids = gp.GeoDataFrame(centroids)

In [24]:

m = folium.Map([41.8781, -87.6298], zoom_start=11, tiles = 'Stamen Toner')

bins = np.nanquantile(chi_wac['TOTAL_EMP_PER_SQ_MILE'], [0, 0.50,0.75,0.98,1])
print(bins.astype(int))


folium.Choropleth(
         geo_data=chi_tracts,
         name= 'Jobs',
         data=chi_wac,
         columns = ['CENSUS_TRA', 'TOTAL_EMP_PER_SQ_MILE'],
         key_on='feature.properties.TRACT',
         fill_color='RdPu',
         fill_opacity=0.6,
         line_opacity=0.2,
         legend_name='Total Jobs Per Square Mile',
         highlight = True,
         bins = bins,
            ).add_to(m)

bins = np.nanquantile(chi_rac['TOTAL_WORKERS_PER_SQ_MILE'], [0, 0.50,0.75,0.98,1])
print(bins.astype(int))

folium.Choropleth(
         geo_data=chi_tracts,
         name= 'Workers',
         data=chi_rac,
         columns = ['CENSUS_TRA', 'TOTAL_WORKERS_PER_SQ_MILE'],
         key_on='feature.properties.TRACT',
         fill_color='RdPu',
         fill_opacity=0.6,
         line_opacity=0.2,
         legend_name='Total Workers Per Square Mile',
         highlight = True,
         bins = bins,
            ).add_to(m)


feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)

for tract2 in centroids.GEOID:
    row = centroids[centroids['GEOID'] == tract2]
    folium.CircleMarker([row['Y'], row['X']], popup = str(int(row['GEOID'].values[0])), radius = 1, fill = True, fill_color = 'grey', color = 'grey').add_to(feature_group)

m.add_child(feature_group)

folium.LayerControl().add_to(m)
m.save('C:/Workspace/TNC-Demand-Model/Data Exploration/LEHD/LEHD 2019 Data.html')

print('Everything is Complete!')

[    15   2103   5753  81636 782984]
[    75   6491  11499  42075 137723]
Everything is Complete!
